In [4]:
import sys
assert sys.version_info >= (3, 8)

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from utils import display_images, display_image

if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

# Weekly activity
1. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.
2. Use the images with titles: _"flower.jfif"_ and _"native-bee.png"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html) for more info. The result should resemble the following:  
![bee and flowers](img_embed/activity3.PNG "bee_flower")
3. Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```

4. Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [10]:
# 1
import imutils
image = cv.imread("images/lena.jfif")
#strategy 1: use imutils library
imutils_rotation = imutils.rotate(image, 45)
cv.imshow("imutils", imutils_rotation)

#stategy 2: modify the transformation matrix
h, w = image.shape[:2]
center = ( w // 2, h // 2)
angle = 45
scale = 1
M = cv.getRotationMatrix2D(center, angle, scale)

dst = cv.warpAffine(image, M, (w, h))
cv.imshow("transformation", dst)
cv.waitKey(0)
cv.destroyAllWindows()

In [12]:
# 2
flower_img = cv.imread("images/flower.jfif")
bee_img = cv.imread("images/native-bee.png")

# location of flower on top of bee's head
rows, cols, channels = flower_img.shape
roi = bee_img[0:rows, 0:cols]

# mask of flower and create inverse mask of flower
flower_gray = cv.cvtColor(flower_img, cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(flower_gray, 74, 255, cv.THRESH_BINARY)
inv_mask = cv.bitwise_not(mask)

# black-out the area of flower in ROI
bee_bg = cv.bitwise_and(roi, roi, mask = inv_mask)

# extract only region of flower from the flower_img
flower_fg = cv.bitwise_and(flower_img, flower_img, mask = mask)

# put flower in ROI and modify the main image
dst = cv.add(bee_bg, flower_fg)
bee_img[0:rows, 0:cols] = dst

# Display result
cv.imshow('Result', bee_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [30]:
# 3
def random_center_crop(image, min_crop_ratio, max_crop_ratio):
    
    crop_ratio = np.random.uniform(min_crop_ratio, max_crop_ratio)

    crop_size = int(min(image.shape[0], image.shape[1]) * crop_ratio)
    
    x = (image.shape[1] - crop_size) // 2
    y = (image.shape[0] - crop_size) // 2
    
    cropped_image = image[y:y + crop_size, x:x + crop_size]

    return cropped_image
    
img = cv.imread("images/dice.jfif")
cropped_img = random_center_crop(img, 0.5, 1)
cv.imshow("original images", img)
cv.imshow("random center cropped image", cropped_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [177]:
# 4 Salt-and-pepper noise
def snp_noise(image):
    rols, cols, ch = image.shape
    svp = 0.5
    amount = 0.5
    image_noise = np.copy(image)

    num_salt = np.ceil(amount * image.size * svp)
    cd = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    image_noise[tuple(cd)] = 255

    # Pepper mode
    num_pepper = np.ceil(amount* image.size * (1. - svp))
    cd = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    image_noise[tuple(cd)] = 0
    return image_noise

img = cv.imread("images/lena.jfif")

img_noise = snp_noise(img)

cv.imshow("noise image", img_noise)
cv.imshow("ori image", img)
cv.waitKey(0)
cv.destroyAllWindows()